In [1]:
# Imports
import os
import requests
import json

from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
load_dotenv()
os.environ["OOPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
MODEL = "gpt-4o-mini"

openai = OpenAI()

In [12]:
# A class for a Webpage

class Website:
  """ 
  A utility class to use to scrap a Website.
  """

  url: str
  title: str
  body: str
  links: List[str]
  text: str

  def __init__(self, url):
    """Create a Website object from the given url using the BeautifulSoup library."""

    self.url = url
    response = requests.get(url)

    self.body = response.content
    soup = BeautifulSoup(markup=self.body, features="html.parser")
    
    self.title = soup.title.string if soup.title else "No title found"

    if self.body:

      for irrelevant in soup.body(["script", "style", "img", "input"]):
        irrelevant.decompose()
      self.text = soup.body.get_text(separator="\n", strip=True)

    else:
      self.text = ""

    soup_links = [link.get('href') for link in soup.find_all('a')]
    self.links = [link for link in soup_links if link]

  def get_contents(self):
    return f'Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n'

In [13]:
sample_website = Website("https://edwarddonner.com")
sample_website.get_contents(), sample_website.links

('Webpage Title:\nHome - Edward Donner\nWebpage Contents:\nHome\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,\nacquired in 2021\n.\nWe work with groundbreaking, proprietary LLMs verticalized for talent, we’ve\npatented\nour matching model, and our award-winning platform has happy c

### 1. Have GPT-4o-mini figure ou which links are relevant


In [15]:
link_system_prompt = """You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Job pages. You should respond in JSON as in this example:
{
  "links" : [
    {"type": "about page", "url": "https://full.url/goes/here/about"},    
    {"type": "careers page", "url": "https://another.full.url/careers"}
  ]
}
"""

In [22]:
def get_links_user_prompt(website):
  return f"Here are the list of links on the website of {website.url}. Please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.\nLinks (some might relevant links):\n{"\n".join(website.links)}"

In [23]:
aws_website =  Website("https://aws.amazon.com/")
print(get_links_user_prompt(aws_website))

Here are the list of links on the website of https://aws.amazon.com/. Please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might relevant links):
#aws-page-content-main
https://aws.amazon.com/?nc2=h_lg
/about-aws/?nc2=h_header
/contact-us/?nc2=h_header
#
#
#
https://console.aws.amazon.com/console/home?nc2=h_ct&src=header-signin
https://portal.aws.amazon.com/gp/aws/developer/registration/index.html?nc2=h_ct&src=header_signup
/q/?nc2=h_ql_prod_l1_q
/products/?nc2=h_ql_prod
/solutions/?nc2=h_ql_sol
/pricing/?nc2=h_ql_pr
https://aws.amazon.com/documentation-overview/?nc2=h_ql_doc_do
/getting-started/?nc2=h_ql_le
/partners/?nc2=h_ql_pn
https://aws.amazon.com/marketplace/?nc2=h_ql_mp
/customer-enablement/?nc2=h_ql_ce
/events/?nc2=h_ql_ev
/contact-us/?nc2=h_ql_exm
https://aws.amazon.com/ar/?nc1=h_ls
https://aws.amazon.com/id/?nc1=h_ls
https://aws.

In [ ]:
# Combining all the codes above
def get_links(url):
  website = Website(url)
  completion = openai.chat.completions.create(
    model=MODEL,
    messages=[
      {"role": "system", "content": link_system_prompt},
      {"role": "user", "content": get_links_user_prompt(website)}
    ],
    response_format={"type":"json_object"}
  )

  result = completion.choices[0].message.content
  return json.loads(result)

In [27]:
get_links("https://aws.amazon.com/")

{'links': [{'type': 'about page',
   'url': 'https://aws.amazon.com/about-aws/?nc2=h_header'},
  {'type': 'careers page', 'url': 'https://aws.amazon.com/careers/'},
  {'type': 'contact page',
   'url': 'https://aws.amazon.com/contact-us/?nc2=h_header'},
  {'type': 'getting started page',
   'url': 'https://aws.amazon.com/getting-started/?nc2=h_ql_le'},
  {'type': 'training page',
   'url': 'https://aws.amazon.com/training/digital/?hp=tc2'},
  {'type': 'solutions page',
   'url': 'https://aws.amazon.com/solutions/?hp=c8#Industry'},
  {'type': 'partners page',
   'url': 'https://aws.amazon.com/partners/?sc_icampaign=aware_apn_recruit&sc_ichannel=ha&sc_icontent=awssm-evergreen-apn_recruit-default-editorial-lower&sc_iplace=ed&trk=bb2355f7-5720-4484-afb3-7cdf3be6ea8a~ha_awssm-evergreen-apn_recruit-default-editorial-lower'},
  {'type': 'documentation page',
   'url': 'https://aws.amazon.com/documentation-overview/?nc2=h_ql_doc_do'},
  {'type': 'case studies page',
   'url': 'https://aws.amaz

### 2. Make a brochure


In [28]:
def get_all_link_details(url:str):

  # Initial website or domain
  website = Website(url)
  result = f"Landing page:\n{website.get_contents()}"

  # Getting the relevant links from OpenAI API
  completion = openai.chat.completions.create(
    model=MODEL,
    messages=[
      {"role": "system", "content": link_system_prompt},
      {"role": "user", "content": get_links_user_prompt(website)}
    ],
    response_format={"type":"json_object"}
  )

  # Respond of OpenAI API as a relevant links of the target website
  links = json.loads(completion.choices[0].message.content)

  # Printing the list of links
  print("Found links:", links)

  # Getting more details about the links
  for link in links["links"]:
    url_page_content = Website(link["url"]).get_contents()
    result += f"\n\n{link["type"]}\n{url_page_content}"

  return result

In [29]:
print(get_all_link_details("https://anthropic.com"))

Found links: {'links': [{'type': 'home page', 'url': 'https://anthropic.com/'}, {'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}, {'type': 'api page', 'url': 'https://anthropic.com/api'}, {'type': 'pricing page', 'url': 'https://anthropic.com/pricing'}, {'type': 'enterprise page', 'url': 'https://anthropic.com/enterprise'}, {'type': 'news page', 'url': 'https://anthropic.com/news'}]}
Landing page:
Webpage Title:
Home \ Anthropic
Webpage Contents:
Claude
Overview
Team
Enterprise
API
Pricing
Research
Company
Careers
News
AI
research
and
products
that put safety at the frontier
Claude.ai
Meet Claude 3.5 Sonnet
Claude 3.5 Sonnet, our most intelligent AI model, is now available.
Talk to Claude
API
Build with Claude
Start using Claude to drive efficiency and create new revenue streams.
L

In [30]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website and creates a short brochure about the company for prospective customers, investors and recruiters. Respond in markdown. Include details of the company culture, customers and careers/jobs if you have the information."

In [32]:
def get_brochure_user_prompt(company_name, url):

  # Initializing the user prompt
  user_prompt = f"You are looking at a company called: {company_name}\n Here are the contents of its landing page and other relevant pages; use this information to buioild a short brochure of the company in markdown.\n"

  # Getting the links and detailed contents of the target website.
  user_prompt += get_all_link_details(url)[:20_000] # Limit the content of the user prompt to 20,000 characters

  # Returning the user prompt
  return user_prompt

In [ ]:
def create_brochure(company_name, url):

  # Calling OpenAI API 
  response = openai.chat.completions.create(
    model=MODEL,
    messages=[
      {"role": "system", "content":system_prompt},
      {"role": "user", "content":get_brochure_user_prompt(company_name, url)}
    ]
  )

  # Getting the response
  result = response.choices[0].message.content
  display(Markdown(result))

In [34]:
create_brochure("Amazon Web Services", "https://aws.amazon.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://aws.amazon.com/about-aws/?nc2=h_header'}, {'type': 'careers page', 'url': 'https://aws.amazon.com/careers/'}, {'type': 'contact page', 'url': 'https://aws.amazon.com/contact-us/?nc2=h_mobile'}, {'type': 'solutions page', 'url': 'https://aws.amazon.com/solutions/?hp=c8#Industry'}, {'type': 'partners page', 'url': 'https://aws.amazon.com/partners/?sc_icampaign=aware_apn_recruit&sc_ichannel=ha&sc_icontent=awssm-evergreen-apn_recruit-default-editorial-lower&sc_iplace=ed&trk=bb2355f7-5720-4484-afb3-7cdf3be6ea8a~ha_awssm-evergreen-apn_recruit-default-editorial-lower'}, {'type': 'products page', 'url': 'https://aws.amazon.com/products/?hp=wacs1'}, {'type': 'documentation page', 'url': 'https://aws.amazon.com/documentation-overview/?nc2=h_ql_doc_do'}, {'type': 'training and certification page', 'url': 'https://aws.amazon.com/training/digital/?hp=tc2'}, {'type': 'marketplace page', 'url': 'https://aws.amazon.com/marketplace/?sc_icam

# Amazon Web Services (AWS) Brochure

## About AWS

Amazon Web Services (AWS) is the world’s most comprehensive and widely adopted cloud platform, offering over 200 fully featured services from data centers globally. Since its inception in 2006, AWS has transformed the way businesses operate by providing the infrastructure necessary to build solutions that can innovate and adapt to changing market dynamics.

### Mission and Vision
AWS strives to be Earth’s most customer-centric company. By working backwards from customer needs, AWS empowers businesses—ranging from startups to enterprises and government agencies—to leverage cloud technology that enables agility, cost reduction, and rapid innovation.

## Company Culture

At AWS, **innovation is the core of our culture**. Every employee is viewed as a "builder," which embodies a proactive, Day One mentality that drives continuous improvement and discovery. Our inclusive culture encourages diverse opinions and perspectives, embracing the belief that different voices foster better solutions and innovation.

### Inclusion and Diversity
AWS is dedicated to creating a workplace that values and promotes inclusion, diversity, and equity. We believe in the strength that diverse backgrounds bring to problem-solving and team dynamics.

### Work-Life Balance
We support hybrid working models to prioritize employee well-being, allowing our team members to balance their professional and personal lives while significantly impacting the world through their work.

## Customer Success

AWS serves millions of active customers, including the fastest-growing startups, largest enterprises, and leading government agencies. Our customers across various industries—from healthcare to finance—benefit from secure, scalable cloud infrastructures that adapt to their needs.

### ROI for Customers
Research shows that organizations moving to AWS reported a remarkable 162% return on investment within three years. This reflects the robust capabilities of AWS, including its comprehensive AI and data services, which catalyze business performance.

## Careers at AWS

AWS is rapidly expanding and continuously seeks talented individuals ready to join our mission to change the world through cloud computing.

### Opportunities for Growth
- **Empowered Teams**: Employees are encouraged to take ownership of their projects, driving innovation while working with smart, passionate colleagues across the globe.
- **Global Presence**: With offices in key locations worldwide, including Arlington, Singapore, Tel Aviv, and Tokyo, opportunities for collaboration and growth are abundant.
- **Learning and Development**: AWS offers diverse training programs and resources to help employees build the skills needed to thrive in a fast-paced tech environment.

### Join Us
AWS is on the lookout for innovators, disruptors, and forward-thinkers who are excited about making a difference. Explore opportunities to become part of our diverse team and contribute to cutting-edge projects.

## Why Choose AWS?

- **Unmatched Operational Expertise**: With the largest network of data centers and a proven history of dependable service, AWS delivers unparalleled reliability, security, and performance critical for mission-critical workloads.
- **Broadest Cloud Capabilities**: AWS provides the deepest set of cloud services tailored to meet varied business needs, empowering organizations of all sizes to adopt cloud technology with ease.

## Get Started with AWS

Whether you're an organization aiming to optimize your operations or a developer keen to build globally scalable solutions, AWS welcomes you to explore and experience the cloud that drives transformation.

**Contact us today or create an AWS account to start building!**

---

For more information:
- Visit [AWS Website](https://aws.amazon.com)
- Explore career opportunities at [AWS Careers](https://aws.amazon.com/careers)

**AWS: Powering customer innovation and transforming industries with the most secure and comprehensive cloud environment.**

### Finally, just a minor improvement

Adding a typewritter animation just link OpenAI website


In [35]:
def stream_brochure(company_name, url):
  stream = openai.chat.completions.create(
    model = MODEL,
    messages=[
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
    ],
    stream=True
  )

  response=""
  display_handle = display(Markdown(""), display_id=True)

  # Loop in each stream
  for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)

In [36]:
stream_brochure("Amazon Web Services", "https://aws.amazon.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://aws.amazon.com/about-aws/?nc2=h_header'}, {'type': 'careers page', 'url': 'https://aws.amazon.com/careers/'}, {'type': 'contact page', 'url': 'https://aws.amazon.com/contact-us/?nc2=h_header'}, {'type': 'solutions page', 'url': 'https://aws.amazon.com/solutions/?hp=c8#Industry'}, {'type': 'partners page', 'url': 'https://aws.amazon.com/partners/?hp=wacs2'}, {'type': 'training page', 'url': 'https://aws.amazon.com/training/digital/?hp=tc2'}, {'type': 'certification page', 'url': 'https://aws.amazon.com/certification/?hp=tc3'}, {'type': 'analyst reports page', 'url': 'https://aws.amazon.com/resources/analyst-reports/?sc_icampaign=aware_analyst_reports&sc_ichannel=ha&sc_icontent=awssm-evergreen-analyst_aware&sc_iplace=ed&trk=a38f3ed5-b313-4b99-8781-ae29a193662c~ha_awssm-evergreen-analyst_aware'}, {'type': 'global infrastructure page', 'url': 'https://aws.amazon.com/about-aws/global-infrastructure/?hp=wacs5'}]}


# Amazon Web Services (AWS) Brochure

## About AWS
Amazon Web Services (AWS) is the world's most comprehensive and broadly adopted cloud platform, launching in 2006 with the mission to empower organizations and individuals by providing innovative cloud solutions. As part of Amazon, AWS stands by its commitment to being Earth's most customer-centric company, architecting its services to meet the evolving needs of its diverse clientele.

### Our Impact
AWS technologies provide a wide spectrum of services that enhance the ability for businesses, non-profits, and governments to address complex challenges, transform industries, and foster community growth.

## Products and Solutions
AWS offers a broad array of cloud services and solutions designed for flexibility, scalability, and security. These include:

- **Compute Power**: Virtual servers with Amazon EC2 and Amazon Lightsail.
- **Storage**: Scalable object storage with Amazon S3.
- **Databases**: Managed NoSQL solutions like Amazon DynamoDB.
- **Artificial Intelligence**: Tools for building generative AI applications with Amazon Bedrock.
- **Networking**: High-speed content delivery via Amazon CloudFront.

Customers across every industry, including financial services, healthcare, telecommunications, and gaming sectors, leverage AWS to drive innovation and efficiency.

## Customer Success
Millions of active customers—from the fastest-growing startups to large enterprises and public sector organizations—use AWS to innovate more quickly, reduce costs, and scale rapidly. AWS empowers its clients to reinvent their businesses, drive transformative change, and succeed in their respective fields.

## Company Culture
At AWS, every individual is encouraged to be a builder. The company fosters a culture where innovative thinking is celebrated, and workers are empowered to drive their projects. AWS promotes a diverse and equitable workplace where every voice matters, contributing to a rich tapestry of ideas and skills. 

### Diversity, Inclusion, and Equity
AWS is committed to building an inclusive workforce that reflects the diversity of its global customer base. The company believes that varied perspectives lead to better problem-solving and solutions.

## Careers at AWS
AWS is actively seeking talented individuals who are eager to be part of a revolutionary tech landscape. With a flexible work environment that emphasizes hybrid schedules, AWS allows its staff to balance personal and professional commitments. 

- **Career Opportunities**: AWS boasts a global presence with roles in various domains, from software engineering and machine learning research to cloud security and data analytics.
- **Learning and Development**: AWS provides resources for continual learning through AWS Digital Classrooms and certifications, encouraging employees to build new skills and knowledge.

### Join Us
Are you ready to change the future of tech with AWS? Explore current open roles and join our talent network to stay updated on career opportunities.

## Connect with Us
To learn more about AWS, explore AWS Marketplace, or access detailed product documentation, visit [AWS Official Site](https://aws.amazon.com).

---

For prospective customers, investors, and recruiters, AWS represents a unique opportunity to engage with one of the most innovative companies in the technology space, leading the charge in cloud solutions and services globally.

In [39]:
stream_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}]}


# Anthropic Company Brochure

---

## About Us

**Anthropic** is an AI safety and research company based in San Francisco. We are focused on developing **reliable, interpretable,** and **steerable AI systems** that can positively impact various spheres of society. With an interdisciplinary team comprised of experts in machine learning, physics, policy, and business, we tackle both the **opportunities** and **risks** associated with artificial intelligence. 

### Our Mission
At Anthropic, we believe that AI will significantly influence the world. Our purpose is to create AI systems that people can trust and to promote safety and reliability throughout the industry.

## Our Products

- **Claude 3.5 Sonnet**: The latest iteration of our AI platform, offering enhanced capabilities for enterprise integration.
- **API Access**: Helps businesses drive efficiency and unlock new revenue streams using our AI technology.

## Our Customers

We serve a diverse clientele, including businesses, nonprofits, and civil society organizations. Our tools empower teams across various sectors to improve productivity, generate high-quality outputs, and collaborate effectively.

## Company Culture

At Anthropic, our values shape our work and define our culture:

1. **Here for the Mission**: We prioritize the responsible development and deployment of AI for the benefit of society.
2. **Unusually High Trust**: We foster a workplace where honesty and constructive disagreement are valued.
3. **One Big Team**: Collaboration is key. Everyone contributes to our shared goals across diverse functions.
4. **Do the Simple Thing That Works**: We embrace pragmatic and empirical approaches to problem-solving. 

### Team Structure
Our team consists of researchers, engineers, policy experts, and operations leaders who collaborate towards common objectives. We are committed to continuous learning and sharing insights with the world about AI safety.

## Careers at Anthropic

We are always looking for passionate individuals to join our mission of building safe AI systems. Here’s what you can expect from a career at Anthropic:

### What We Offer
- **Health & Wellness**: Comprehensive health, dental, and vision insurance, inclusive fertility benefits, and generous paid parental leave.
- **Compensation**: Competitive salary and equity packages, retirement plans, and life protection plans.
- **Work Environment**: Flexible paid time off, daily meals, and a supportive office atmosphere.

### Hiring Process
Our interview process is designed to minimize bias and assess a candidate's fit thoroughly. It typically includes:
1. Resume submission
2. Exploratory chat
3. Skills assessment (role-dependent)
4. Team screen and culture interview
5. Final checks and offer

We value diversity and don’t require formal degrees or specific prior experience in machine learning for technical roles.

## Join Us!

If you want to be part of a forward-thinking team dedicated to safe and responsible AI development, [explore our open positions](#careers) and become an integral piece of the Anthropic puzzle.

---

For more information, updates, and insights, follow us on [Twitter](#), [LinkedIn](#), and [YouTube](#).

***Contact Us:***  
For inquiries or support, visit our [contact page](#).  

---

© 2024 Anthropic PBC. All rights reserved.

## Business Applications

In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

In terms of techniques, this is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

In terms of business applications - generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.
